# Data selection for NDVI Calculation

In this notebook, we will select the tiles that we want to process with the workflow. 

In [3]:
import os
import pickle
import asyncio
import ipywidgets as widgets
import ibm_boto3
import ibm_botocore

from datetime import date
from pprint import pprint
from ipyleaflet import *
from sentinelsat import SentinelAPI, geojson_to_wkt

Set the IBM Cloud Object Storage and Sentinel-2 credentials below.

For Sentinel-2, you can register and access data for free at https://sentinel.esa.int/web/sentinel/sentinel-data-access/registration:

In [4]:
# IBM COS Credentials
IBM_API_KEY_ID='<IBM_API_KEY_ID>'
IBM_SERVICE_INSTANCE_ID='<IBM_SERVICE_INSTANCE_ID>'
ENDPOINT_URL='<ENDPOINT_URL>'
BUCKET='<BUCKET>'

# SENTINEL API Credentials
SENTINEL_USERNAME='<SENTINEL_USERNAME>'
SENTINEL_PASSWORD='<SENTINEL_PASSWORD>'

### Download Sentinel 2 Data

Choose the date range in which you want to download Sentinel-2 tiles:

In [20]:
from_day = widgets.DatePicker(
    description='From day',
    disabled=False,
    value=date(2019, 11, 3)
) 
to_day = widgets.DatePicker(
    description='To day',
    disabled=False,
    value=date(2019, 11, 10)
)

display(from_day)
display(to_day)

DatePicker(value=datetime.date(2019, 11, 3), description='From day')

DatePicker(value=datetime.date(2019, 11, 10), description='To day')

Select the maximum cloud percentage a tile is allowed to have:

In [15]:
percentage = widgets.IntSlider(
    value=15,
    min=0,
    max=100,
    step=1,
    description='Porcentaje de nubosidad',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
percentage

IntSlider(value=15, continuous_update=False, description='Porcentaje de nubosidad')

Select (left click) on the following map the range of tiles you want to get from Sentinel-2 (right click erases the current selection):

In [16]:
m = Map(center=(37.9, -1.4), zoom=9)

polygon = Polygon(
        locations=[[]],
        color="green",
        fill_color="green"
    )

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        pol = next(layer for layer in m.layers if isinstance(layer, Polygon))
        coords = kwargs.get('coordinates')
        if (len(polygon.locations) == 0):
            pol.locations[0].extend([coords, coords])
        else:
            pol.locations[0].insert(1, coords)
    
        m.remove_layer(pol)
        other = Polygon(
            locations=pol.locations,
            color="green",
            fill_color="green"
        )
        m.add_layer(other)
        
    if kwargs.get('type') == 'contextmenu':
        pol = next(layer for layer in m.layers if isinstance(layer, Polygon))
        m.remove_layer(pol)
        other = Polygon(
            locations=[[]],
            color="green",
            fill_color="green"
        )
        m.add_layer(other)

m.on_interaction(handle_click)
m.add_layer(polygon);
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Here we collect the selected shape:

In [17]:
locations = [[]]
first = None
for layer in m.layers:
    if isinstance(layer, Polygon):
        locations[0] = [[loc[1], loc[0]] for loc in layer.locations[0]]
        
if (len(locations[0]) > 0):
    locations[0].append(locations[0][0])

locations

[[[-1.485516203645365, 37.134319076375355],
  [-0.679519855755555, 37.786046441392266],
  [-1.0036876737897285, 38.48167913134617],
  [-1.036653076662182, 38.64060672038247],
  [-1.127307934561399, 38.737079966071335],
  [-1.4047667420711398, 38.681355735600185],
  [-1.500915833782426, 38.550447699956685],
  [-1.470697547816027, 38.38055624080693],
  [-1.60805339311787, 38.30515999687283],
  [-1.745355750941027, 38.38055624080693],
  [-2.4689840560066227, 38.09592994354168],
  [-1.485516203645365, 37.134319076375355]]]

Set date range, cloud percentage and area parameters:

In [21]:
from_date =  from_day.value 
to_date = to_day.value
cloudcoverpercentage=(0, percentage.value)
array_area = locations[0]
geo_json_area = {
  "features": [
    {
      "geometry": {
        "coordinates": [
          array_area
        ],
        "type": "Polygon"
      },
      "properties": {
        
      },
      "type": "Feature"
    }
  ],
  "type": "FeatureCollection"
}

Get metadata from Sentinel-2:

In [22]:
api = SentinelAPI(user=SENTINEL_USERNAME,
                  password=SENTINEL_PASSWORD)

footprint = geojson_to_wkt(geo_json_area)
products = api.query(footprint,
                     date=(from_date, to_date),
                     platformname='Sentinel-2',
                     producttype=('S2MS2Ap', 'S2MSI1C'),
                     cloudcoverpercentage=cloudcoverpercentage)

print('Products: {}'.format(len(products)))

Products: 5


Upload tiles metadata to COS

In [23]:
client_config = ibm_botocore.client.Config(signature_version='oauth',
                                       max_pool_connections=128)
cos = ibm_boto3.client('s3',
                       ibm_api_key_id=IBM_API_KEY_ID,
                       ibm_service_instance_id=IBM_SERVICE_INSTANCE_ID,
                       config=client_config,
                       endpoint_url=ENDPOINT_URL)
keys = []

for key, value in products.items():
    obj = pickle.dumps(value)
    key = key + '.bin'
    cos.put_object(Bucket=BUCKET, Key=key, Body=obj)   
    keys.append(key)

'["94db240c-5ff7-4324-bcec-f03f9c54295d.bin", "5de8810b-5464-41b8-b9f8-0205dec825eb.bin", "a621faa0-d9fe-4d6f-8a6e-54ace7619f07.bin", "e383811c-7161-4e47-8683-1f579a4a1616.bin", "feafa8e0-db9f-4e01-9799-1e7b73af3335.bin"]'

In [27]:
print(json.dumps(keys))

["94db240c-5ff7-4324-bcec-f03f9c54295d.bin", "5de8810b-5464-41b8-b9f8-0205dec825eb.bin", "a621faa0-d9fe-4d6f-8a6e-54ace7619f07.bin", "e383811c-7161-4e47-8683-1f579a4a1616.bin", "feafa8e0-db9f-4e01-9799-1e7b73af3335.bin"]


Put the previous JSON output to an Airflow variable with key SENTINEL_TILES:

![title](images/airflow_vars.png)

Don't forget to upload the `shapefile.zip` file to COS, needed by some tasks of the workflow.

In [28]:
cos.upload_file(Filename='shapefile.zip', Bucket=BUCKET, Key='shapefile.zip')

Now you are ready to execute the workflow:

![title](images/airflow_dag.png)